In [37]:
$target = 'TCSValidate'
$institution = 'University of Western States'
# Set the encoding to UTF-8 for the sf cli responses
[Console]::OutputEncoding = [System.Text.Encoding]::UTF8

Set-Location -Path (Get-Location)
$path = "Purge"
if (-not (Test-Path -Path $path)) {
    New-Item -Path $path -ItemType Directory | Out-Null
    Write-Host "Directory created: $path"
} else {
    Write-Host "Directory already exists: $path"
}


# Run the sf data query and capture the institution AccountID
$records = sf data query -q "SELECT Id FROM Account WHERE Name='$institution' AND Type='Partner'" --target-org $target --json | ConvertFrom-Json
$insAccountID= $records.result.records[0].Id
Write-Host "Found Id: $insAccountID"


Directory already exists: Purge
 »   Warning: @salesforce/cli update available from 2.82.6 to 2.84.6.
Found Id: 001D400000v2ihaIAA


### Record Purge

Purges records from the target instance, in opposite order of the dependency tree.
Indiscremenately purges all records of the specified type, regardless of the state of the record.

#### SObjects
- Account
- Location
- Address
- Learning
- LearningProgram
- Academic Term
- ProgramTermApplnTimeline
- Scholarship__c
- IndividualApplication
- Opportunity
- Scholarship_Award__c
- Task
- Event

In [14]:
#Delete Events 
$predicate = " accountId in(SELECT Id FROM Account where School__c='$institution')"
sf data query -q "SELECT Id FROM Event   WHERE $predicate" --target-org $target --json | Out-File -FilePath Purge\Event.json -Encoding UTF8
$records = Get-Content -Raw -Path Purge\Event.json | ConvertFrom-Json

$ids = $records.result.records  | Select-Object -ExpandProperty Id
if (-not $ids) {
    Write-Host "Events - No IDs found."
    return
} 

$ids | ForEach-Object { [PSCustomObject]@{ Id = $_ } } | Export-Csv -Path Purge\Event.csv -NoTypeInformation
sf data delete bulk --sobject "Event" --file Purge\Event.csv --target-org $target --json --wait 15 | Out-File -FilePath Purge\DeletedEvent.json -Encoding UTF8
 

 »   Warning: @salesforce/cli update available from 2.82.6 to 2.84.6.
No IDs found.


In [15]:
#Delete Tasks 
$predicate = " accountId in(SELECT Id FROM Account where School__c='$institution')"
sf data query -q "SELECT Id FROM Task   WHERE $predicate" --target-org $target --json | Out-File -FilePath Purge\Task.json -Encoding UTF8
$records = Get-Content -Raw -Path Purge\Task.json | ConvertFrom-Json
$ids = $records.result.records  | Select-Object -ExpandProperty Id
if (-not $ids) {
    Write-Host "Tasks - No IDs found."
    return
} 
$ids | ForEach-Object { [PSCustomObject]@{ Id = $_ } } | Export-Csv -Path Purge\Task.csv -NoTypeInformation
sf data delete bulk --sobject "Task" --file Purge\Task.csv --target-org $target --json --wait 15 | Out-File -FilePath Purge\DeletedTask.json -Encoding UTF8

 »   Warning: @salesforce/cli update available from 2.82.6 to 2.84.6.
Tasks - No IDs found.


In [16]:
#Delete Scholarship_Award__c 
$predicate = " Award_Institution__c='$institution'"
sf data query -q "SELECT Id FROM Scholarship_Award__c  WHERE $predicate" --target-org $target --json | Out-File -FilePath Purge\Scholarship_Award__c.json -Encoding UTF8
$records = Get-Content -Raw -Path Purge\Scholarship_Award__c.json | ConvertFrom-Json
$ids = $records.result.records  | Select-Object -ExpandProperty Id
if (-not $ids) {
    Write-Host "Scholarship_Award__c - No IDs found."
    return
} 
$ids | ForEach-Object { [PSCustomObject]@{ Id = $_ } } | Export-Csv -Path Purge\Scholarship_Award__c.csv -NoTypeInformation
sf data delete bulk --sobject "Scholarship_Award__c" --file Purge\OpportScholarship_Award__cunity.csv --target-org $target --json --wait 15 | Out-File -FilePath Purge\DeletedScholarship_Award__c.json -Encoding UTF8

 »   Warning: @salesforce/cli update available from 2.82.6 to 2.84.6.
Scholarship_Award__c - No IDs found.


In [17]:
#Delete Opportunity 
$predicate = " account.School__c='$institution'"
sf data query -q "SELECT Id FROM Opportunity   WHERE $predicate" --target-org $target --json | Out-File -FilePath Purge\Opportunity.json -Encoding UTF8
$records = Get-Content -Raw -Path Purge\Opportunity.json | ConvertFrom-Json
$ids = $records.result.records  | Select-Object -ExpandProperty Id
if (-not $ids) {
    Write-Host "Opportunity - No IDs found."
    return
} 
$ids | ForEach-Object { [PSCustomObject]@{ Id = $_ } } | Export-Csv -Path Purge\Opportunity.csv -NoTypeInformation
sf data delete bulk --sobject "Opportunity" --file Purge\Opportunity.csv --target-org $target --json --wait 15 | Out-File -FilePath Purge\DeletedOpportunity.json -Encoding UTF8

. {
>> #Delete Opportunity 
>> $predicate = " account.School__c='$institution'"
>> sf data query -q "SELECT Id FROM Opportunity   WHERE $predicate" --target-org $target --json | Out-File -FilePath Purge\Opportunity.json -Encoding UTF8
>> $records = Get-Content -Raw -Path Purge\Opportunity.json | ConvertFrom-Json
>> $ids = $records.result.records  | Select-Object -ExpandProperty Id
>> if (-not $ids) {
>>     Write-Host "Opportunity - No IDs found."
>>     return
>> } 
>> $ids | ForEach-Object { [PSCustomObject]@{ Id = $_ } } | Export-Csv -Path Purge\Opportunity.csv -NoTypeInformation
>> sf data delete bulk --sobject "Opportunity" --file Purge\Opportunity.csv --target-org $target --json --wait 15 | Out-File -FilePath Purge\DeletedOpportunity.json -Encoding UTF8
>> }
>> 
 »   Warning: @salesforce/cli update available from 2.82.6 to 2.84.6.
Opportunity - No IDs found.


In [18]:
#Delete IndividualApplication
$predicate = " OpportunityId IN (SELECT Id FROM Opportunity where account.School__c='$institution')"
sf data query -q "SELECT Id FROM IndividualApplication  WHERE $predicate" --target-org $target --json | Out-File -FilePath Purge\IndividualApplication.json -Encoding UTF8
$records = Get-Content -Raw -Path Purge\IndividualApplication.json | ConvertFrom-Json
$ids = $records.result.records  | Select-Object -ExpandProperty Id
if (-not $ids) {
    Write-Host "IndividualApplication - No IDs found."
    return
} 
$ids | ForEach-Object { [PSCustomObject]@{ Id = $_ } } | Export-Csv -Path Purge\IndividualApplication.csv -NoTypeInformation
sf data delete bulk --sobject "IndividualApplication" --file Purge\IndividualApplication.csv --target-org $target --json --wait 15 | Out-File -FilePath Purge\DeletedIndividualApplication.json -Encoding UTF8

 »   Warning: @salesforce/cli update available from 2.82.6 to 2.84.6.
IndividualApplication - No IDs found.


In [19]:
#Delete Scholarship__c
$predicate = " School_Name__c='$institution'"
sf data query -q "SELECT Id FROM Scholarship__c  WHERE $predicate" --target-org $target --json | Out-File -FilePath Purge\Scholarship__c.json -Encoding UTF8
$records = Get-Content -Raw -Path Purge\Scholarship__c.json | ConvertFrom-Json
$ids = $records.result.records  | Select-Object -ExpandProperty Id
if (-not $ids) {
    Write-Host "Scholarship__c - No IDs found."
    return
} 
$ids | ForEach-Object { [PSCustomObject]@{ Id = $_ } } | Export-Csv -Path Purge\Scholarship__c.csv -NoTypeInformation
sf data delete bulk --sobject "Scholarship__c" --file Purge\Scholarship__c.csv --target-org $target --json --wait 15 | Out-File -FilePath Purge\DeletedScholarship__c.json -Encoding UTF8

 »   Warning: @salesforce/cli update available from 2.82.6 to 2.84.6.
Scholarship__c - No IDs found.


In [ ]:
#Delete ProgramTermApplnTimeline
$predicate = " Provider_Name__c='$institution'"
sf data query -q "SELECT Id FROM ProgramTermApplnTimeline  WHERE $predicate" --target-org $target --json | Out-File -FilePath Purge\ProgramTermApplnTimeline.json -Encoding UTF8
$records = Get-Content -Raw -Path Purge\ProgramTermApplnTimeline.json | ConvertFrom-Json
$ids = $records.result.records  | Select-Object -ExpandProperty Id
if (-not $ids) {
    Write-Host "ProgramTermApplnTimeline - No IDs found."
    return
} 
$ids | ForEach-Object { [PSCustomObject]@{ Id = $_ } } | Export-Csv -Path Purge\ProgramTermApplnTimeline.csv -NoTypeInformation
sf data delete bulk --sobject "ProgramTermApplnTimeline" --file Purge\ProgramTermApplnTimeline.csv --target-org $target --json --wait 15 | Out-File -FilePath Purge\DeletedProgramTermApplnTimeline.json -Encoding UTF8

In [ ]:
#Delete AcademicTerm

sf data query -q "select Id FROM AcademicTerm where Institution__c='$insAccountID'" --target-org $target --json | Out-File -FilePath Purge\AcademicTerm.json -Encoding UTF8
$records = Get-Content -Raw -Path Purge\AcademicTerm.json | ConvertFrom-Json
$ids = $records.result.records  | Select-Object -ExpandProperty Id
if (-not $ids) {
    Write-Host "AcademicTerm - No IDs found."
    return
} 
$ids | ForEach-Object { [PSCustomObject]@{ Id = $_ } } | Export-Csv -Path Purge\AcademicTerm.csv -NoTypeInformation
sf data delete bulk --sobject "AcademicTerm" --file Purge\AcademicTerm.csv --target-org $target --json --wait 15 | Out-File -FilePath Purge\DeletedAcademicTerm.json -Encoding UTF8


In [ ]:
#Delete LearningProgram
$predicate = " Provider_Name__c='$institution'"
sf data query -q "SELECT Id FROM LearningProgram  WHERE $predicate" --target-org $target --json | Out-File -FilePath Purge\LearningProgram.json -Encoding UTF8
$records = Get-Content -Raw -Path Purge\LearningProgram.json | ConvertFrom-Json
$ids = $records.result.records  | Select-Object -ExpandProperty Id
if (-not $ids) {
    Write-Host "LearningProgram - No IDs found."
    return
} 
$ids | ForEach-Object { [PSCustomObject]@{ Id = $_ } } | Export-Csv -Path Purge\LearningProgram.csv -NoTypeInformation
sf data delete bulk --sobject "LearningProgram" --file Purge\LearningProgram.csv --target-org $target --json --wait 15 | Out-File -FilePath Purge\LearningProgram.json -Encoding UTF8

In [ ]:
#Delete Learning

sf data query -q "select Id FROM Learning where ProviderId='$insAccountID'" --target-org $target --json | Out-File -FilePath Purge\Learning.json -Encoding UTF8
$records = Get-Content -Raw -Path Purge\Learning.json | ConvertFrom-Json
$ids = $records.result.records  | Select-Object -ExpandProperty Id
if (-not $ids) {
    Write-Host "Learning - No IDs found."
    return
} 
$ids | ForEach-Object { [PSCustomObject]@{ Id = $_ } } | Export-Csv -Path Purge\Learning.csv -NoTypeInformation
sf data delete bulk --sobject "Learning" --file Purge\Learning.csv --target-org $target --json --wait 15 | Out-File -FilePath Purge\DeletedLearning.json -Encoding UTF8


In [ ]:

#Delete Address

sf data query -q "select ID FROM Address where ParentId in (select ID from Location where Provider__c='$insAccountID')" --target-org $target --json | Out-File -FilePath Purge\Address.json -Encoding UTF8
$records = Get-Content -Raw -Path Purge\Address.json | ConvertFrom-Json
$ids = $records.result.records  | Select-Object -ExpandProperty Id
if (-not $ids) {
    Write-Host "Address - No IDs found."
    return
} 
$ids | ForEach-Object { [PSCustomObject]@{ Id = $_ } } | Export-Csv -Path Purge\Address.csv -NoTypeInformation
sf data delete bulk --sobject "Address" --file Purge\Address.csv --target-org $target --json --wait 15 | Out-File -FilePath Purge\DeletedAddress.json -Encoding UTF8


In [ ]:
#Delete Location

sf data query -q "select ID FROM Location where Provider__c='$insAccountID'" --target-org $target --json | Out-File -FilePath Purge\Location.json -Encoding UTF8
$records = Get-Content -Raw -Path Purge\Location.json | ConvertFrom-Json
$ids = $records.result.records  | Select-Object -ExpandProperty Id
if (-not $ids) {
    Write-Host "Location - No IDs found."
    return
} 
$ids | ForEach-Object { [PSCustomObject]@{ Id = $_ } } | Export-Csv -Path Purge\Location.csv -NoTypeInformation
sf data delete bulk --sobject "Location" --file Purge\Location.csv --target-org $target --json --wait 15 | Out-File -FilePath Purge\DeletedLocation.json -Encoding UTF8


In [ ]:
# #Deleteing Refrence objects using Migration\results folder  
# @(
#      "ProgramTermApplnTimeline",
#      "AcademicTerm",
#      "LearningProgram",
#       "Learning",
#       "Address",
#       "Location"
#      ) | ForEach-Object {
# $records = Get-Content -Raw -Path results\$($_).json | ConvertFrom-Json
# $ids = $records.result.records.successfulResults  | Select-Object -ExpandProperty sf__Id
# #sf__Id
# $ids | ForEach-Object { [PSCustomObject]@{ Id = $_ } } | Export-Csv -Path Purge\$s$($_).csv -NoTypeInformation
# sf data delete bulk --sobject $($_) --file Purge\$($_).csv --target-org $target --json --wait 15 | Out-File -FilePath Purge\$($_).json -Encoding UTF8
# }

 »   Warning: @salesforce/cli update available from 2.82.6 to 2.84.6.
 »   Warning: @salesforce/cli update available from 2.82.6 to 2.84.6.
 »   Warning: @salesforce/cli update available from 2.82.6 to 2.84.6.
 »   Warning: @salesforce/cli update available from 2.82.6 to 2.84.6.
 »   Warning: @salesforce/cli update available from 2.82.6 to 2.84.6.
 »   Warning: @salesforce/cli update available from 2.82.6 to 2.84.6.


In [21]:
#Delete Account 
$predicate = " School__c='$institution'"
sf data query -q "SELECT Id FROM Account WHERE $predicate" --target-org $target --json | Out-File -FilePath Purge\Account.json -Encoding UTF8
$records = Get-Content -Raw -Path Purge\Account.json | ConvertFrom-Json
$ids = $records.result.records  | Select-Object -ExpandProperty Id
if (-not $ids) {
    Write-Host "Account - No IDs found."
    return
} 
$ids | ForEach-Object { [PSCustomObject]@{ Id = $_ } } | Export-Csv -Path Purge\Account.csv -NoTypeInformation
sf data delete bulk --sobject "Account" --file Purge\Account.csv --target-org $target --json --wait 15 | Out-File -FilePath Purge\DeletedAccount.json -Encoding UTF8

 »   Warning: @salesforce/cli update available from 2.82.6 to 2.84.6.
 »   Warning: @salesforce/cli update available from 2.82.6 to 2.84.6.


In [ ]:

#  #Deleteing  objects using bindings
# @(
# #"Event", 
#     #"Task", 
#     #"Scholarship_Award__c", 
#     #"Opportunity", 
#     #"IndividualApplication",
#     #"Scholarship__c", 
#     "ProgramTermApplnTimeline",
#     "AcademicTerm",
#     "LearningProgram",
#     "Learning",
#     "Location"# delete location and address 
#      ) | ForEach-Object {
# $records = Get-Content -Raw -Path Purge\bindings.json | ConvertFrom-Json
#  $ids = $records.$($_)  | Select-Object -ExpandProperty TargetId
#  if ($ids.totalSize -eq 0) {
#      return
#  }

# $ids | ForEach-Object { [PSCustomObject]@{ Id = $_ } } | Export-Csv -Path Purge\$s$($_).csv -NoTypeInformation
# # sf data delete bulk --sobject $($_) --file Purge\$($_).csv --target-org $target --json --wait 15 | Out-File -FilePath Purge\$($_).json -Encoding UTF8
#  }


In [ ]:
# Clean up
Remove-Item -Path Purge -Recurse -Force